In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
pandas  1.0.4
seaborn 0.10.1
numpy   1.18.4



In [38]:
inf_truth = "/mnt/nvidia/pkr/code/type_1/data/mock_communities/bmock_12/SRR8073716/bmock_12_truth.txt"
inf_tax_file = "/mnt/btrfs/data/rep202/ab/taxmap.tsv"
outf_truth = "/mnt/nvidia/pkr/code/type_1/data/mock_communities/bmock_12/SRR8073716/bmock_12_truth_rg.txt"

inf_coverage = "/mnt/nvidia/pkr/code/type_1/data/mock_communities/bmock_12/SRR8073716/features_test.csv"

In [39]:
with open(outf_truth, "w") as outf:
    with open(inf_truth) as inf:
        for query in inf:
            query = query.rstrip()
            print(query)
            results = !rg {query} {inf_tax_file}
            for result in results:
                outf.write(result + "\n")

Muricauda
Thioclava
Cohaesibacter
Propionibacteriaceae
Marinobacter
Marinobacter
Psychrobacter
Micromonospora_echinaurantiaca
Micromonospora_echinofusca
Micromonospora_coxensis
Halomonas
Halomonas


In [40]:
df_features = pd.read_csv(inf_coverage, index_col=0)

In [41]:
df_tax = pd.read_csv(inf_tax_file, names=["assembly_accession", "tax"], sep="\t")

df_tax["species"] = [_.split(";")[6] for _ in df_tax.tax]
df_features = pd.merge(df_features, df_tax, how="left", on="assembly_accession")



In [42]:
df_truth = pd.read_csv(outf_truth, header=None, sep="\t", names=["assembly_accession", "tax"])
s_truth = set(df_truth["assembly_accession"])

df_features["truth"] = [_ in s_truth for _ in df_features.assembly_accession]

In [43]:
df_features["truth"].sum()
len(s_truth)

225

In [44]:
df_features["truth"].value_counts()

False    7027
True      172
Name: truth, dtype: int64

In [45]:
len(s_truth)

225

In [47]:
df_features.loc[~df_features["truth"]].sort_values("hits")

,assembly_accession,hits,percent_coverage,expected_coverage,percent_padded_coverage,shannon_entropy,percent_max_uncovered_region,largest_pileup,gc_content,genome_length,num_n_groups,consecutive_ns,tax,species,truth
7198,GCF_004362525.1,1,6.436516e+11,0.000010,2.027502e+11,0.000179,0.999990,1,0.664227,6367892,35,100,k__Bacteria;p__Proteobacteria;c__Betaproteobac...,s__Paucibacter_toxinivorans,False
5633,GCF_001687365.1,1,5.377327e+11,0.000012,1.962724e+11,0.000208,0.999988,1,0.610246,6265340,30,20,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s__Pararhizobium_polonicum,False
6389,GCF_001816165.1,1,1.772978e+11,0.000024,8.953537e+10,0.000401,0.999976,1,0.605338,4231675,132,20,k__Bacteria;p__Bacteroidetes;c__Cytophagia;o__...,s__Hymenobacter_glacialis,False
6394,GCF_003049665.1,1,3.714504e+11,0.000014,1.411512e+11,0.000251,0.999986,1,0.366605,5313213,137,20,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,s__Lysinibacillus_mangiferihumi,False
6395,GCF_900167335.1,1,1.019420e+11,0.000027,3.873797e+10,0.000453,0.999973,1,0.379747,2783450,85,1123,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s__Pilibacter_termitis,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,GCF_000027145.1,2367943,6.437916e+07,1.000000,3.996919e+07,NaN,0.020309,14896,0.373826,2797636,0,0,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,s__Listeria_seeligeri,False
2,GCF_000763515.1,2389722,6.742681e+07,1.000000,4.399042e+07,NaN,0.028136,15887,0.371513,2919548,0,0,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,s__Listeria_ivanovii,False
6,GCF_000006765.1,3974842,6.538959e+06,1.000000,6.494966e+06,NaN,0.003377,488,0.665557,6264404,0,0,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Pseudomonas_aeruginosa,False
5,GCF_009648575.1,5408760,1.538624e+07,1.000000,1.052119e+07,NaN,0.019713,14896,0.374763,2922148,1,20,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,s__Listeria_innocua,False


In [51]:
len(set(df_features["species"]))

7196

In [48]:
df_features.loc[df_features["truth"]].sort_values("hits")

,assembly_accession,hits,percent_coverage,expected_coverage,percent_padded_coverage,shannon_entropy,percent_max_uncovered_region,largest_pileup,gc_content,genome_length,num_n_groups,consecutive_ns,tax,species,truth
6390,GCF_900111365.1,1,8.050749e+10,0.000035,3.944867e+10,0.000567,0.999965,1,0.668563,2808867,58,83,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,s__Propionibacterium_cyclohexanicum,True
6338,GCF_004126535.1,1,1.459264e+11,0.000021,4.523719e+10,0.000351,0.999979,1,0.676919,3007896,25,20,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,s__Brevilactibacter_flavus,True
6198,GCF_000428985.1,1,inf,0.000000,1.916830e+11,-0.000000,2.000000,0,0.564151,4378162,68,1403,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Marinobacterium_litorale,True
5267,GCF_900103865.1,1,2.599361e+11,0.000016,8.577891e+10,0.000277,0.999984,1,0.558682,4141954,38,175,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Halomonas_arcis,True
6393,GCF_003798325.1,2,4.817298e+10,0.000064,2.360476e+10,0.000981,0.826250,1,0.705891,3072768,0,0,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,s__Propionibacterium_acidifaciens,True
6391,GCF_900637915.1,2,4.728547e+10,0.000064,2.316988e+10,0.000989,0.969817,1,0.683029,3044331,0,0,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,s__Propionibacterium_australiense,True
5515,GCF_004916895.1,2,2.228868e+11,0.000030,6.654914e+10,0.000304,0.999983,2,0.423212,3747248,145,20,k__Bacteria;p__Bacteroidetes;c__Flavobacteriia...,s__Muricauda_alvinocaridis,True
6392,GCF_000940845.1,2,3.580653e+10,0.000074,1.754520e+10,0.001122,0.825149,1,0.673358,2649166,0,0,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,s__Propionibacterium_freudenreichii,True
3838,GCF_000372805.1,3,8.497760e+10,0.000064,3.252229e+10,0.000724,0.999856,2,0.546971,3629260,144,2866,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Marinobacter_gelidimuriae,True
4317,GCF_000714545.1,3,1.922132e+11,0.000058,7.231784e+10,0.000347,0.999980,3,0.602765,3822068,104,20,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s__Thioclava_indica,True


In [17]:
df_features.groupby("truth").max()

,assembly_accession,hits,percent_coverage,expected_coverage,percent_padded_coverage,shannon_entropy,percent_max_uncovered_region,largest_pileup,gc_content,genome_length,num_n_groups,consecutive_ns
truth,,,,,,,,,,,,
False,GCF_903652925.1,16055403,inf,1.000000,1.540674e+12,0.181698,2.0,15887,0.770192,13661541,1774,4435411
True,GCF_900637915.1,4146,inf,0.118949,2.202497e+11,0.005964,2.0,3853,0.739338,7240126,521,13695


In [ ]:
import lightgbm

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
X = df_masked.loc[:, df_masked.columns.difference(["assembly_accession", "tax", "truth", "species"])]
y = df_masked["truth"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#
# Create the LightGBM data containers
#
# categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
# train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [ ]:
#
# Train the model
#

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

lgb = lightgbm.LGBMClassifier(**parameters)
lgb.fit(X_train, y_train)

In [ ]:
lgb_disp = plot_roc_curve(lgb, X_test, y_test)
plt.show()

In [ ]:
sns.pairplot(df_masked, hue="truth")

In [ ]:
import joblib

In [ ]:
rfc_hmp_mock = joblib.load("/mnt/nvidia/pkr/code/type_1/data/hmp_mock_community/shi7_learning_tminx/rfc.pkl")

In [ ]:
rfc_disp = plot_roc_curve(rfc_hmp_mock, X, y)

In [ ]:
predictions = rfc_hmp_mock.predict_proba(X)

In [ ]:
(predictions[:, 1] > .75).sum()

In [ ]:
X[predictions[:, 1] > .75]

In [ ]:
for i in [.1, .2, .3, .4, .5, .6, .7, .8, .9]:
    print(y[predictions[:, 1] > i].sum() / (predictions[:, 1] > i).sum())
print(df_masked.loc[predictions[:, 1] > .5, ["tax", "truth"]].values)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

X = df_masked.loc[:, df_masked.columns.difference(["truth", "tax"])]
y = df_masked["truth"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

svc_disp = plot_roc_curve(svc, X_test, y_test)
rfc_disp = plot_roc_curve(rfc, X_test, y_test, ax=svc_disp.ax_)
rfc_disp.figure_.suptitle("ROC curve comparison")

plt.show()